# Instalação de dependências
Executar esta célula caso ocorra erro de "ModuleNotFoundError"

In [ ]:
%load_ext autoreload
%autoreload 2
%pip install webdriver-manager selenium openai python-dotenv beautifulsoup4 yfinance anthropic 

In [ ]:
PROVIDER = "claude" 
#PROVIDER = "openai"

# Configuração Inicial do Driver
Este driver será reutilizado por todos os módulos. **Não feche o navegador!**

In [ ]:
from automation.driver import create_driver
from automation.config import DOWNLOADS_PUBLIC, DOWNLOADS_PRIVATE
from selenium.common.exceptions import WebDriverException
import os

def get_or_create_driver(current_driver=None):
    if current_driver:
        try:
            # Tenta acessar uma propriedade para ver se a sessão ainda existe
            current_driver.current_url
            return current_driver
        except WebDriverException:
            print("Sessão anterior inválida. Criando novo driver...")
    
    return create_driver()

# Inicializa ou recupera o driver
if 'driver' not in locals():
    driver = None

driver = get_or_create_driver(driver)

# Download HTML relatórios Suno

In [ ]:
from automation.suno.auth import login_suno
from automation.suno.reports import download_suno_reports

driver = get_or_create_driver(driver)
login_suno(driver)
download_suno_reports(driver, os.path.join(DOWNLOADS_PUBLIC, "html-relatorios"))

# Resumos Relatórios Suno - OpenAI

In [ ]:
from automation.analysis.processors import process_reports

process_reports(
    html_folder=os.path.join(DOWNLOADS_PUBLIC, "html-relatorios"),
    output_folder=os.path.join(DOWNLOADS_PUBLIC, "resumos-relatorios"),
    provider=PROVIDER
)

# Extração Carteiras Suno

In [ ]:
from automation.suno.auth import login_suno
from automation.suno.wallets import download_suno_wallets

driver = get_or_create_driver(driver)
login_suno(driver)
download_suno_wallets(driver, os.path.join(DOWNLOADS_PUBLIC, "html-carteiras"))

# Carteiras Suno em .md e CSV - OpenAI

In [ ]:
from automation.analysis.processors import process_suno_wallets_step1_html_to_md, process_suno_wallets_step2_md_to_csv

html_folder = os.path.join(DOWNLOADS_PUBLIC, "html-carteiras")
md_folder = os.path.join(DOWNLOADS_PUBLIC, "html-carteiras-csv")
csv_folder = os.path.join(DOWNLOADS_PUBLIC, "carteiras-csv")

process_suno_wallets_step1_html_to_md(html_folder, md_folder)
process_suno_wallets_step2_md_to_csv(md_folder, csv_folder)

# Meus Dividendos Table em HTML

In [ ]:
from automation.meus_dividendos.scraper import download_meus_dividendos_wallet

driver = get_or_create_driver(driver)
download_meus_dividendos_wallet(driver, os.path.join(DOWNLOADS_PRIVATE, "meus-dividendos"))

# Meus Dividendos Table em CSV

In [ ]:
from automation.analysis.processors import process_meus_dividendos_to_csv
from automation.config import DOWNLOADS_PRIVATE
import os
from datetime import date

today_str = date.today().strftime("%Y-%m-%d")
html_file = os.path.join(DOWNLOADS_PRIVATE, "meus-dividendos", f"carteira-meus-dividendos-{today_str}.htm")
csv_folder = os.path.join(DOWNLOADS_PRIVATE, "meus-dividendos-csv")

process_meus_dividendos_to_csv(html_file, csv_folder)